**The purpose of this notebook is to test the performance of the Imagededup library on train set.**

Thanks to @Tanay's notebook [Finding similar Images using Image Deduplicator 💻](https://www.kaggle.com/heyytanay/finding-similar-images-using-image-deduplicator) and dataset [imagededup-cp37](https://www.kaggle.com/heyytanay/imagededupcp37).

#### Step 1 : Installation & Import

In [ ]:
! pip install --no-deps "../input/imagededupcp37/imagededup-0.2.2-cp37-cp37m-manylinux1_x86_64.whl"

In [ ]:
import numpy as np
import pandas as pd
from tqdm import tqdm
tqdm.pandas()

import imagededup
from imagededup.methods import CNN, PHash, DHash, WHash, AHash

#### Step 2 : Use hashing method to match duplicates

In [ ]:
# %%time
# cnn = CNN()
# encodings = cnn.encode_images(image_dir="../input/shopee-product-matching/train_images")
# duplicates = cnn.find_duplicates(encoding_map=encodings)

# # CV = 0.235

In [ ]:
# %%time
# hashing_method = PHash()
# encodings = hashing_method.encode_images(image_dir="../input/shopee-product-matching/train_images")
# duplicates = hashing_method.find_duplicates(encoding_map=encodings)

# # CV = 0.192

In [ ]:
# %%time
# hashing_method = DHash()
# encodings = hashing_method.encode_images(image_dir="../input/shopee-product-matching/train_images")
# duplicates = hashing_method.find_duplicates(encoding_map=encodings)

# # CV = 0.166

In [ ]:
# %%time
# hashing_method = WHash()
# encodings = hashing_method.encode_images(image_dir="../input/shopee-product-matching/train_images")
# duplicates = hashing_method.find_duplicates(encoding_map=encodings)

# # CV = 0.081

In [ ]:
%%time
hashing_method = AHash()
encodings = hashing_method.encode_images(image_dir="../input/shopee-product-matching/train_images")
duplicates = hashing_method.find_duplicates(encoding_map=encodings)

# CV = 0.09

#### Step 3 : Inference result process

In [ ]:
train = pd.read_csv('../input/shopee-product-matching/train.csv')

In [ ]:
def getDuplicates(img_id):
    dups = duplicates[img_id]
    index = train[train.image.isin(dups)].index
    return train.iloc[index].posting_id.values

train['preds'] = train['image'].progress_apply(getDuplicates)

#### Step 4 : Evaluation

In [ ]:
def getMetric(col):
    def f1score(row):
        n = len( np.intersect1d(row.target,row[col]) )
        return 2*n / (len(row.target)+len(row[col]))
    return f1score

def combine_for_cv(row):
    x = np.concatenate([row.preds])
    return np.unique(x)

tmp = train.groupby('label_group').posting_id.agg('unique').to_dict()
train['target'] = train.label_group.map(tmp)
train['oof'] = train.apply(combine_for_cv, axis=1)
train['f1'] = train.apply(getMetric('oof'), axis=1)
print('CV Score =', round(train.f1.mean(), 3) )

#### Conclusion : so far its performance is not very satisfactory.

GitHub : [imagededup](https://github.com/idealo/imagededup)